In [5]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile
class soundSimulator:
    def __init__(self,total_time = 5):
        """the intensity is proportional to amplitude squared"""
        """the intensity is 1/distance**2"""
        """the amplitude is then inverse proportional to distance"""
        self.time = 0 #time will be discretized by the sampling rate..
        self.total_time = 5
        self.x_bound = [0,12]
        self.y_bound = [0,8]
        self.sample_rate = 22050
        fs,self.s1 = scipy.io.wavfile.read( 'data/music/'+str(1)+'.wav')
        fs,self.s2 = scipy.io.wavfile.read( 'data/speech/'+str(1)+'.wav') 
    def agentMovement(self,t):
        agent1 = [1,1]
        agent2 = [2,2]
        return agent1,agent2
    
    def startSimulate(self):
        #At each simulation frame sampled at 22050, calculate when the sound from our source file will reach
        #The target mic. Because it will not reach the mic at the exact sampling frame. some interpolation between 
        #2 source sound point will be needed.
        return
        

In [6]:
a = soundSimulator()
##expected use:
##Assume the position detection happen at 20Hz
dt = 0.05
total_simulation_time = 20
current_time = 0
while (current_time <= total_simulation_time):
    

